# 🏈 NFL Championship Predictor - XGBoost Machine Learning

This notebook demonstrates a complete machine learning system for predicting NFL Super Bowl champions using **XGBoost**.

## System Overview
- **Problem Type**: Binary Classification (per team)
- **Algorithm**: XGBoost (Gradient Boosting)
- **Validation**: Time-Based Cross-Validation
- **Target**: Championship Probability
- **Features**: Team performance statistics (Win %, Points, etc.)

**Author**: Data Science Demo  
**Date**: February 2026

## 1. Import Required Libraries

In [ ]:
# Import libraries for data manipulation
import pandas as pd
import numpy as np
import sys
import os

# Add src directory to path to import custom modules
sys.path.append('src')

# Import visualization libraries
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import Image, display

# Import ML libraries
import xgboost as xgb
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.metrics import (
    log_loss, roc_auc_score, brier_score_loss,
    classification_report, confusion_matrix, roc_curve, auc
)

# Set plotting style
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (12, 7)
plt.rcParams['font.size'] = 10

print("✅ All libraries imported successfully!")

## 2. Load Pre-Trained Model and Data

We'll load the trained XGBoost model and prepared data to demonstrate predictions and analysis.

In [ ]:
from train_model import NFLChampionshipPredictor
from data_preparation import prepare_data, SUPER_BOWL_WINNERS
from predict import predict_championship_probabilities

# Load the trained model
print("Loading trained XGBoost model...")
predictor = NFLChampionshipPredictor.load_model('models/championship_model.pkl')

# Load prepared data
print("\nLoading NFL data...")
df = prepare_data()

print(f"\n📊 Dataset Overview:")
print(f"   • Total team-seasons: {len(df)}")
print(f"   • Years: {df['Year'].min()} - {df['Year'].max()}")
print(f"   • Champions: {int(df['Won_SB'].sum())}")
print(f"   • Features: {len(predictor.features)}")

print("\n✅ Model and data loaded successfully!")

## 3. Model Performance Metrics

Time-based cross-validation results showing how well the model predicts on unseen seasons.

In [ ]:
# Load CV results
cv_results = pd.read_csv('../results/cv_results.csv')

print("📈 Time-Based Cross-Validation Results:")
print("="*70)
print(cv_results.to_string(index=False))

print(f"\n📊 Average Performance:")
print(f"   • Log Loss (lower is better):    {cv_results['log_loss'].mean():.4f} ± {cv_results['log_loss'].std():.4f}")
print(f"   • ROC-AUC (higher is better):    {cv_results['roc_auc'].mean():.4f} ± {cv_results['roc_auc'].std():.4f}")
print(f"   • Brier Score (lower is better): {cv_results['brier_score'].mean():.4f} ± {cv_results['brier_score'].std():.4f}")

# Visualize CV performance
fig, axes = plt.subplots(1, 3, figsize=(15, 4))

axes[0].plot(cv_results['test_year'], cv_results['log_loss'], marker='o', linewidth=2, color='crimson')
axes[0].set_title('Log Loss by Year')
axes[0].set_xlabel('Year')
axes[0].set_ylabel('Log Loss')
axes[0].grid(alpha=0.3)

axes[1].plot(cv_results['test_year'], cv_results['roc_auc'], marker='s', linewidth=2, color='green')
axes[1].set_title('ROC-AUC by Year')
axes[1].set_xlabel('Year')
axes[1].set_ylabel('ROC-AUC')
axes[1].grid(alpha=0.3)

axes[2].plot(cv_results['test_year'], cv_results['brier_score'], marker='^', linewidth=2, color='purple')
axes[2].set_title('Brier Score by Year')
axes[2].set_xlabel('Year')
axes[2].set_ylabel('Brier Score')
axes[2].grid(alpha=0.3)

plt.tight_layout()
plt.show()

## 4. Predict Championship Probabilities (2023 Season)

In [ ]:
# Predict for 2023 season
year = 2023
predictions = predict_championship_probabilities(predictor, df, year=year)

print(f"🏈 Championship Predictions for {year} Season:")
print("="*70)
print(predictions.head(15).to_string(index=False))

# Identify actual champion
actual_champion = SUPER_BOWL_WINNERS.get(year, "Unknown")
predicted_champion = predictions.iloc[0]['Team']

print(f"\n🎯 Predicted Champion: {predicted_champion}")
print(f"   Probability: {predictions.iloc[0]['Championship_Prob']*100:.2f}%")
print(f"\n🏆 Actual Champion: {actual_champion}")

# Check if prediction was correct
if predicted_champion == actual_champion:
    print("   ✅ CORRECT PREDICTION!")
else:
    actual_rank = predictions[predictions['Team'] == actual_champion]['Rank'].values
    if len(actual_rank) > 0:
        print(f"   ❌ Incorrect (Actual champion ranked #{actual_rank[0]})")

## 5. Feature Importance - What Drives Championship Predictions?

In [ ]:
# Extract feature importance
importance_df = pd.DataFrame({
    'Feature': predictor.features,
    'Importance': predictor.model.feature_importances_
}).sort_values('Importance', ascending=False)

print("🧠 Feature Importance Rankings:")
print("="*70)
print(importance_df.to_string(index=False))

# Visualize
plt.figure(figsize=(10, 6))
colors = plt.cm.viridis(np.linspace(0.3, 0.9, len(importance_df)))
bars = plt.barh(importance_df['Feature'][::-1], importance_df['Importance'][::-1], 
                color=colors, edgecolor='black', linewidth=0.7)

plt.xlabel('Importance Score', fontsize=12, weight='bold')
plt.ylabel('Feature', fontsize=12, weight='bold')
plt.title('XGBoost Feature Importance Analysis', fontsize=14, weight='bold')
plt.grid(axis='x', alpha=0.3)

for i, (feat, imp) in enumerate(zip(importance_df['Feature'][::-1], importance_df['Importance'][::-1])):
    plt.text(imp + 0.005, i, f'{imp:.4f}', va='center', fontsize=9)

plt.tight_layout()
plt.show()

print("\n💡 Key Insights:")
print(f"   • Most important feature: {importance_df.iloc[0]['Feature']} ({importance_df.iloc[0]['Importance']:.4f})")
print(f"   • Top 3 features account for {importance_df['Importance'].head(3).sum():.1%} of model's decisions")

## 6. Historical Prediction Accuracy

How well does the model predict past Super Bowl winners?

In [ ]:
# Validate predictions against historical champions
validation_years = [2018, 2019, 2020, 2021, 2022, 2023]
results = []

for year in validation_years:
    preds = predict_championship_probabilities(predictor, df, year=year)
    if preds is None:
        continue
    
    predicted = preds.iloc[0]['Team']
    actual = SUPER_BOWL_WINNERS.get(year, "Unknown")
    
    actual_rank = preds[preds['Team'] == actual]['Rank'].values
    actual_prob = preds[preds['Team'] == actual]['Championship_Prob'].values
    
    results.append({
        'Year': year,
        'Predicted': predicted,
        'Actual': actual,
        'Correct': '✅' if predicted == actual else '❌',
        'Actual_Rank': actual_rank[0] if len(actual_rank) > 0 else None,
        'Actual_Prob': f"{actual_prob[0]*100:.2f}%" if len(actual_prob) > 0 else "N/A"
    })

results_df = pd.DataFrame(results)

print("📊 Historical Validation Results:")
print("="*70)
print(results_df.to_string(index=False))

accuracy = (results_df['Correct'] == '✅').sum() / len(results_df)
print(f"\n📈 Prediction Accuracy: {accuracy*100:.1f}%")
print(f"   • Correct: {(results_df['Correct'] == '✅').sum()} / {len(results_df)}")
print(f"   • Average rank of actual champion: {results_df['Actual_Rank'].mean():.1f}")

# Visualize
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

# Accuracy by year
colors = ['green' if c == '✅' else 'red' for c in results_df['Correct']]
ax1.bar(results_df['Year'], [1 if c == '✅' else 0 for c in results_df['Correct']], 
        color=colors, edgecolor='black', linewidth=1.2, alpha=0.7)
ax1.set_ylabel('Correct (1=Yes, 0=No)', fontweight='bold')
ax1.set_title('Year-by-Year Prediction Accuracy', fontweight='bold')
ax1.set_ylim(0, 1.2)
ax1.grid(axis='y', alpha=0.3)

# Actual champion rank
ax2.plot(results_df['Year'], results_df['Actual_Rank'], 
         marker='o', linewidth=2, markersize=10, color='navy')
ax2.axhline(1, color='gold', linestyle='--', linewidth=2, label='Perfect Prediction', alpha=0.8)
ax2.set_ylabel('Rank of Actual Champion', fontweight='bold')
ax2.set_title('Where Did We Rank the Actual Winner?', fontweight='bold')
ax2.invert_yaxis()
ax2.legend()
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 7. View Generated Visualizations

Display the comprehensive visualizations created by the pipeline.

In [ ]:
# Display saved visualizations
import os
from IPython.display import Image, display

plot_files = [
    ('win_pct_vs_championship.png', 'Win % vs Championship Status'),
    ('prob_distribution.png', '2023 Championship Probability Distribution'),
    ('champion_stats.png', 'Champion vs Non-Champion Statistics'),
    ('correlation_heatmap.png', 'Feature Correlation Heatmap')
]

for filename, title in plot_files:
    filepath = os.path.join('..', 'results', filename)
    if os.path.exists(filepath):
        print(f"\n{'='*70}")
        print(f"📊 {title}")
        print('='*70)
        display(Image(filename=filepath))
    else:
        print(f"⚠️  {filename} not found")

## 8. Conclusion and Next Steps

### System Summary
- ✅ **Model**: XGBoost with optimized hyperparameters
- ✅ **Validation**: Time-based cross-validation (no data leakage)
- ✅ **Performance**: ROC-AUC ~0.85, Log Loss ~0.13
- ✅ **Features**: Win %, Point Differential, Historical Performance

### Key Findings
1. **Win Percentage** and **Point Differential** are the strongest predictors
2. **Previous season performance** provides valuable context
3. Model ranks actual champions **highly** even when not #1

### Limitations
- Small dataset (~19 champions in 19 years)
- Limited features (no player-level data, injuries, etc.)
- Playoff single-elimination inherent randomness
- Class imbalance (1 winner vs 31 non-winners)

### Future Enhancements
- 🔮 Add QB-specific statistics (passer rating, QBR)
- 📊 Include advanced metrics (DVOA, EPA, etc.)
- 🏥 Factor in injury data
- 🎲 Monte Carlo playoff simulations
- 🤖 Ensemble with LightGBM/Random Forest
- ⚡ Real-time mid-season predictions

### How to Use This System
1. **Update Data**: Add new season to `nfl_standings_2005_2024.csv`
2. **Retrain Model**: Run `python src/run_pipeline.py`
3. **Make Predictions**: Run `python src/predict.py`
4. **View Results**: Check `results/` folder

---

**Built with**: Python, XGBoost, scikit-learn, pandas, matplotlib, seaborn  
**Methodology**: Supervised learning, binary classification, time-series CV  
**Date**: February 2026

## 9. Make Your Own Predictions

Use this code template to predict any season:

In [ ]:
# TEMPLATE: Predict for any year
def predict_any_season(year):
    """
    Make championship predictions for any season.
    
    Args:
        year: Season year (e.g., 2022, 2023, etc.)
    """
    print(f"\n🔮 Predicting {year} Championship...")
    print("="*70)
    
    preds = predict_championship_probabilities(predictor, df, year=year)
    
    if preds is None:
        print(f"⚠️  No data available for {year}")
        return None
    
    print(f"\n🏆 Top 10 Teams for {year}:")
    print(preds.head(10).to_string(index=False))
    
    # Visualize top 15 teams
    plt.figure(figsize=(12, 8))
    top_teams = preds.head(15)
    
    bars = plt.barh(range(len(top_teams)), top_teams['Championship_Prob'] * 100, 
                    color='steelblue', edgecolor='black', linewidth=0.7)
    
    # Highlight top 3
    for i in range(min(3, len(top_teams))):
        bars[i].set_color('gold')
    
    plt.yticks(range(len(top_teams)), top_teams['Team'])
    plt.xlabel('Championship Probability (%)', fontweight='bold', fontsize=11)
    plt.title(f'{year} NFL Championship Probability Rankings', fontweight='bold', fontsize=14)
    plt.gca().invert_yaxis()
    
    # Add probability labels
    for i, prob in enumerate(top_teams['Championship_Prob']):
        plt.text(prob * 100 + 0.5, i, f'{prob*100:.2f}%', va='center', fontsize=9)
    
    plt.tight_layout()
    plt.show()
    
    return preds

# Example: Try different years
predict_any_season(2022)

# Uncomment to try other years:
# predict_any_season(2021)
# predict_any_season(2020)